In [ ]:
import torch
from PIL import Image
import requests
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'retina'
import torchvision.transforms as T
torch.set_grad_enabled(False)

In [ ]:
# COCO classes
CLASSES = [
    'N/A', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus',
    'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'N/A',
    'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse',
    'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'N/A', 'backpack',
    'umbrella', 'N/A', 'N/A', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis',
    'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove',
    'skateboard', 'surfboard', 'tennis racket', 'bottle', 'N/A', 'wine glass',
    'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich',
    'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake',
    'chair', 'couch', 'potted plant', 'bed', 'N/A', 'dining table', 'N/A',
    'N/A', 'toilet', 'N/A', 'tv', 'laptop', 'mouse', 'remote', 'keyboard',
    'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'N/A',
    'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier',
    'toothbrush'
]

# colors for visualization
COLORS = [[0.000, 0.447, 0.741], [0.850, 0.325, 0.098], [0.929, 0.694, 0.125],
          [0.494, 0.184, 0.556], [0.466, 0.674, 0.188], [0.301, 0.745, 0.933]]

In [ ]:
# for output bounding box post-processing
def box_cxcywh_to_xyxy(x):
    x_c, y_c, w, h = x.unbind(1)
    b = [(x_c - 0.5 * w), (y_c - 0.5 * h),
         (x_c + 0.5 * w), (y_c + 0.5 * h)]
    return torch.stack(b, dim=1)

def rescale_bboxes(out_bbox, size):
    img_w, img_h = size
    b = box_cxcywh_to_xyxy(out_bbox)
    b = b * torch.tensor([img_w, img_h, img_w, img_h], dtype=torch.float32)
    return b

def plot_results(pil_img, prob, boxes):
    plt.figure(figsize=(16,10))
    plt.imshow(pil_img)
    ax = plt.gca()
    colors = COLORS * 100
    for p, (xmin, ymin, xmax, ymax), c in zip(prob, boxes.tolist(), colors):
        ax.add_patch(plt.Rectangle((xmin, ymin), xmax - xmin, ymax - ymin,
                                   fill=False, color=c, linewidth=3))
        cl = p.argmax()
        text = f'{CLASSES[cl]}: {p[cl]:0.2f}'
        ax.text(xmin, ymin, text, fontsize=15,
                bbox=dict(facecolor='yellow', alpha=0.5))
    plt.axis('off')
    plt.show()

In [ ]:
model = torch.hub.load('facebookresearch/detr', 'detr_resnet50', pretrained=True)

In [ ]:
model.eval()

In [ ]:
url = 'http://images.cocodataset.org/val2017/000000039769.jpg'

In [ ]:
im = Image.open(requests.get(url, stream=True).raw)

In [ ]:
im

In [ ]:
transform = T.Compose([
  T.Resize(800),
  T.ToTensor(),
  T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [ ]:
img = transform(im).unsqueeze(0) # unsqueeze

In [ ]:
outputs = model(img)

In [ ]:
outputs.keys()

In [ ]:
outputs['pred_logits'].shape

In [ ]:
print(outputs['pred_logits'].softmax(-1).shape)
print(outputs['pred_logits'].softmax(-1))

In [ ]:
probas = outputs['pred_logits'].softmax(-1)[0, :, :-1] # Why removing last element?

In [ ]:
keep = probas.max(-1).values > 0.9

In [ ]:
# convert boxes from [0; 1] to image scales
bboxes_scaled = rescale_bboxes(outputs['pred_boxes'][0, keep], im.size)

In [ ]:
plot_results(im, probas[keep], bboxes_scaled)

## Fine tuning DETR


In [ ]:
#!pip install -q git+https://github.com/huggingface/transformers.git
!pip install transformers
!pip install -q timm

In [ ]:
from transformers import DetrFeatureExtractor, DetrForObjectDetection

In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as T
import timm

In [ ]:
class Detr(nn.Module):
  def __init__(self):
    super().__init__()
    self.feature_extractor = DetrFeatureExtractor.from_pretrained('facebook/detr-resnet-50')
    self.model = DetrForObjectDetection.from_pretrained('facebook/detr-resnet-50')

  def forward(self, x):
    inputs = self.feature_extractor(images=x, return_tensors='pt')
    outputs = self.model(**inputs)
    return outputs.logits, outputs.pred_boxes

## FeatureExtractor test

In [ ]:
feature_extractor = DetrFeatureExtractor.from_pretrained('facebook/detr-resnet-50')

In [ ]:
url = 'http://images.cocodataset.org/val2017/000000039769.jpg'
image = Image.open(requests.get(url, stream=True).raw)

In [ ]:
print(type(image))
print(image.size)

In [ ]:
inputs = feature_extractor(images=image, return_tensors='pt')

In [ ]:
print(inputs.keys())
# Shorter side is rescaled to 800
print(inputs['pixel_values'].shape) # [batch, channel, height, width]
print(inputs['pixel_mask'].shape)

## DetrObjcetDetrForObjectDetection test

In [ ]:
model = DetrForObjectDetection.from_pretrained('facebook/detr-resnet-50')

In [ ]:
outputs = model(**inputs)

In [ ]:
outputs.keys()

In [ ]:
print(f'logits = {outputs.logits.shape}')
print(f'pred_boxes = {outputs.pred_boxes.shape}')
print(f'last_hidden_state = {outputs.last_hidden_state.shape}')
print(f'encoder_last_hidden_state = {outputs.encoder_last_hidden_state.shape}')

## Flickr logos 27 dataset

In [ ]:
from torch.utils.data import Dataset

In [ ]:
class FlickrLogos27(Dataset):
  def __init__(self, annot_file, img_dir, transform=None, target_transform=None):
    pass

  def __len__(self):
    pass

  def __getitem__(self, idx):
    pass